In [1]:
import warnings
import numpy as np
import pandas as pd
from IPython.display import HTML
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly import tools
import matplotlib.pyplot as plt
import statsmodels.api as sm
import datetime
import colorlover as cl
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.neural_network import MLPClassifier, BernoulliRBM
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelBinarizer
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import mean_squared_error, roc_curve, precision_recall_curve
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Input, LSTM, SimpleRNN
from keras.utils import to_categorical
from keras.optimizers import SGD, Adam
from keras.models import Model
from keras import regularizers
from keras import backend as K
import warnings
warnings.filterwarnings('ignore')

/Users/bukowskio/anaconda/lib/python2.7/site-packages/sklearn/utils/fixes.py:313: FutureWarning:

numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.

Using TensorFlow backend.


In [2]:
init_notebook_mode(connected=True)

In [3]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<a href="javascript:code_toggle()">show code</a>.''')

In [4]:
df=pd.read_csv('~/Desktop/energy/final_forecasts.csv')

In [5]:
df2=pd.read_csv('~/Desktop/energy-prices-forecasting/dane.csv')

In [6]:
df3=pd.read_csv("~/Desktop/energy/repo/new_train.csv")

In [7]:
df4=pd.read_csv("~/Desktop/energy/train_data.csv")

# Time-series forecasting - energy prices example

author: Oskar Bukowski

Aim of the project is forecasting market energy prices (on Polish Power Exchange) using machine-learning methods like autoencoders and recurrent neural networks.

##### Data used in forecasting

1. RDN - energy prices in Polish Power Exchange (***price we want to forecast***)
2. CRO - energy prices in balancing markets
3. SE - energy prices in Sweden
4. demand - predicted demand on energy
5. supply - predicted production of powerplants
6. wind_prod - predicted production of windparks
7. reserve - predicted system energy reserve

those features will be further called ***"fundamental features"***

##### Steps
1. features engineering
2. dimensionality reduction using autoencoders
3. forecasting prices in a loop and cross-validation in time

##### Features engineering

New features based on time-series analysis were created:
1. delayed prices : 1-day, 2-days, 3-days, 1-week lags
2. moving averages : 2-days, 3-days, 4-days
3. moving standard deviations : 2-days, 3-days
4. ratios calculated as differences between moving averages and divided by moving standard deviations

those features will be further called ***"time-series features"***


##### Dimensionality reduction usind AEs

Because "time-series features" are highly correlated (with prices and between themselves) and very noisy, dimensionality reduction using autoencoders were used to create smaller, less correlated and denoised dataset


##### Forecasting prices

Data transformed in previous step were used to built models forecasting energy prices:
1. random forests with 200 trees
2. gradient-boosting trees with 200 trees
2. 5 elastic net linear regressions with different penalties
3. 6 feed-forward neural networks with different parameters
4. 3 recurrent neural networks with different parameters

All forecasts were calculated in a loop for every day from test dataset (updating train dataset with every day from test data with every iteration).

# Exploratory analysis

In [8]:
trace1=go.Scatter(
    y=df2['rdn']
)

layout=go.Layout(
    title='energy prices [PLN/MWh] - linear plots',
    autosize=False,
    width=800,
    height=500,
)

data=[trace1]
fig=go.Figure(data=data, layout=layout)

iplot(fig,'plot1', link_text='')

In [9]:
# calculating trend

t=range(len(df2['rdn']))
model0=LinearRegression()
model0.fit(zip(t,t),df2['rdn'])
trend=model0.predict(zip(t,t))

In [10]:
# calculating seasonality

seas_feat=df2['rdn']-trend
seas_feat2=pd.get_dummies(df2['godz'])

model01=LinearRegression()
model01.fit(seas_feat2,df2['rdn'])
seasonality=model01.predict(seas_feat2)

In [11]:
# noise 

noise=df2['rdn']-seasonality

In [12]:
trace1=go.Scatter(
    y=trend,
    line = dict(color = ('rgb(20,200,20)')),
    name='trend'
)
    
trace2=go.Scatter(
    y=seasonality,
    line = dict(color = ('rgb(180,200,20)')),
    name='seasonality'
)
        
trace3=go.Scatter(
    y=noise,
    line = dict(color = ('rgb(180,200,200)')),
    name='fluctuations'
)       

layout=go.Layout(
    title='energy prices - time-series decomposition',
    autosize=False,
    width=800,
    height=700,
)
    
data=[trace1,trace2,trace3]
fig=go.Figure(data=data, layout=layout)

iplot(fig,'plot2', link_text='')

In [13]:
df2.columns

Index([u'Data', u'godz', u'deman', u'supply', u'wind_prod', u'reserve', u'cro',
       u'SE4', u'rdn'],
      dtype='object')

In [14]:
kor1=df2[['rdn','deman','supply','wind_prod','reserve','cro','SE4']].corr()

kols1=kor1.columns

### Correlation matrix within "fundamental data"

In [15]:
trace0=go.Heatmap(
    z=np.array(kor1),
    x=kols1,
    y=kols1
)

layout=go.Layout(
    #title='correlation matrix - "fundamental features"',
    autosize=False,
    width=1000,
    height=600,
)
    
data=[trace0]
fig=go.Figure(data=data, layout=layout)

iplot(fig,'plot3', link_text='')

In [16]:
df4.columns

Index([u'godz', u'deman', u'supply', u'wind_prod', u'reserve', u'cro', u'rdn',
       u'weekend', u'cro2', u'cro3', u'cro4', u'cro5', u'ma_cro2', u'ma_cro3',
       u'sd_cro1', u'sd_cro2', u'ratio_cro1', u'ratio_cro2', u'fix2', u'fix3',
       u'fix4', u'fix5', u'fix6', u'ma_fix2', u'ma_fix3', u'ma_fix4',
       u'sd_fix1', u'sd_fix2', u'se2', u'se3', u'se4'],
      dtype='object')

In [17]:
x4=df4.drop(['godz','deman','supply','wind_prod','reserve','weekend'],axis=1)

In [18]:
kor4=x4.corr()

kols4=kor4.columns

### Correlation matrix within "time-series data"

In [19]:
trace0=go.Heatmap(
    z=np.array(kor4),
    x=kols4,
    y=kols4
)

layout=go.Layout(
    #title='correlation matrix - "time-series features"',
    autosize=False,
    width=1000,
    height=700,
)

data=[trace0]
fig=go.Figure(data=data, layout=layout)

iplot(fig,'plot4', link_text='')

##### Conclusion
We can see that "time-series" features are very strongly correlated

In [20]:
x4.columns

Index([u'cro', u'rdn', u'cro2', u'cro3', u'cro4', u'cro5', u'ma_cro2',
       u'ma_cro3', u'sd_cro1', u'sd_cro2', u'ratio_cro1', u'ratio_cro2',
       u'fix2', u'fix3', u'fix4', u'fix5', u'fix6', u'ma_fix2', u'ma_fix3',
       u'ma_fix4', u'sd_fix1', u'sd_fix2', u'se2', u'se3', u'se4'],
      dtype='object')

In [21]:
df3.columns

Index([u'Unnamed: 0', u'rdn', u'deman', u'supply', u'wind_prod', u'reserve',
       u'0', u'1', u'2', u'3', u'4', u'5', u'6', u'7', u'8', u'9', u'10',
       u'11', u'12', u'13', u'14'],
      dtype='object')

In [22]:
x3=df3.drop(['Unnamed: 0','deman','supply','wind_prod','reserve'],axis=1)

In [23]:
kor3=x3.corr()

kol3=kor3.columns

In [24]:
kol3

Index([u'rdn', u'0', u'1', u'2', u'3', u'4', u'5', u'6', u'7', u'8', u'9',
       u'10', u'11', u'12', u'13', u'14'],
      dtype='object')

### Correlation matrix within data transformed by autoencoders

In [25]:
trace0=go.Heatmap(
    z=np.array(kor3),
    x=kol3,
    y=kol3
)

data=[trace0]
fig=go.Figure(data=data, layout=layout)

iplot(data,'plot4', link_text='')

##### Cocnlusion
We can see that using dimensionality reduction by autoencoders, high correlations within data have been removed

In [26]:
x=df.drop(['Unnamed: 0','real_price'],axis=1)
errors=x.apply(lambda x: x-df['real_price'])

In [27]:
errors2=np.abs(errors)
errors3=pd.DataFrame(errors2).mean(axis=0)

In [28]:
errors3.sort_values()

rnn2      20.735652
rnn3      21.898571
gbt       23.671319
log1      24.151834
rf        24.229770
log4      25.468075
mlp2      25.655700
log3      25.815859
log2      26.178540
arimax    26.565511
rnn1      31.910856
mlp1      32.434337
mlp3      32.591439
mlp6      33.096786
mlp5      34.475001
mlp4      38.551324
dtype: float64

In [29]:
errors3.sort_values()

rnn2      20.735652
rnn3      21.898571
gbt       23.671319
log1      24.151834
rf        24.229770
log4      25.468075
mlp2      25.655700
log3      25.815859
log2      26.178540
arimax    26.565511
rnn1      31.910856
mlp1      32.434337
mlp3      32.591439
mlp6      33.096786
mlp5      34.475001
mlp4      38.551324
dtype: float64

In [30]:
errors3.sort_values().values

array([ 20.73565167,  21.89857129,  23.6713191 ,  24.15183381,
        24.22976979,  25.46807475,  25.65569984,  25.8158591 ,
        26.17853968,  26.56551087,  31.91085645,  32.43433724,
        32.59143892,  33.09678573,  34.47500088,  38.55132429])

In [31]:
error4=errors3/np.mean(df['real_price'])

In [32]:
error4

arimax    0.111273
gbt       0.099151
log1      0.101163
log2      0.109653
log3      0.108133
log4      0.106677
mlp1      0.135856
mlp2      0.107463
mlp3      0.136514
mlp4      0.161478
mlp5      0.144403
mlp6      0.138631
rf        0.101490
rnn1      0.133663
rnn2      0.086854
rnn3      0.091725
dtype: float64

In [33]:
errors4=errors3/np.mean(df['real_price'])

### Forecasting energy prices - results
The plots below show forecast errors of different models used in forecasting energy prices

##### Mean absolute errors of top 10 models

In [34]:
trace1=go.Bar(
    y=errors3.sort_values().values[0:9],
    x=errors3.sort_values().index[0:9]
)

layout=go.Layout(
    #title='MAPE errors',
    autosize=False,
    width=800,
    height=500,
)

data=[trace1]
fig=go.Figure(data=data, layout=layout)

iplot(fig,'plot5', link_text='')

##### Mean absolute percentage errors of top 10 models

In [35]:
trace1=go.Bar(
    y=error4.sort_values().values[0:9],
    x=error4.sort_values().index[0:9]
)

layout=go.Layout(
    #title='MAPE errors',
    autosize=False,
    width=800,
    height=500,
)

data=[trace1]
fig=go.Figure(data=data, layout=layout)

iplot(fig,'plot6', link_text='')

In [38]:
kols=errors3.sort_values().index[0:4]

In [39]:
y=df['real_price']

In [40]:
y_pred=df[kols]

##### Linear plot of price real values and forecasts made by top 4 models

In [64]:
trace1=go.Scatter(
    y=y,
    name='real price',
    line=dict(color='rgb(20,20,200)')
)

trace2=go.Scatter(
    y=np.array(y_pred)[:,0],
    name=kols[0],
    line=dict(color='rgb(200,20,180)')
)

trace3=go.Scatter(
    y=np.array(y_pred)[:,1],
    name=kols[1],
    line=dict(color='rgb(20,200,20)')
)

trace4=go.Scatter(
    y=np.array(y_pred)[:,2],
    name=kols[2],
    line=dict(color='rgb(180,200,20)')
)

trace5=go.Scatter(
    y=np.array(y_pred)[:,3],
    name=kols[3],
    line=dict(color='rgb(200,150,100)')
)

layout=go.Layout(
    #title='MAPE errors',
    autosize=False,
    width=800,
    height=500,
)

data=[trace1,trace2,trace3,trace4,trace5]
fig=go.Figure(data=data, layout=layout)

iplot(data,'forecasts',link_text='')

##### Conclusion:
We can see that only recurrent neural networks were able to predict picks of price.

# Final conclusions:
1. Features engineering using autoencoders can provide better forecast accuracy - it give variety of advantages: dimensionality reduction, removing high correlations, denoising data etc.
2. Recurrent neural networks can outperform other ML methods in forecasting time-series